In [1]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'


In [14]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad, i_sense_digamma_amplin
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)
import src.UsefulFunctions as U
from src.CoreUtil import decimate

In [3]:
def narrow_fit(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.averaged)
    y = np.mean(y[(0, 2), :], axis=0)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [4]:
def narrow_fit_trans_only(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = np.copy(dat.Transition.avg_x)
    y = np.copy(dat.Transition.avg_data)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [5]:
def do_calc(datnum, overwrite=True):
    """Just a function which can be passed to a process pool for faster calculation"""
    save_name = 'SPS.01'

    dat = get_dat(datnum)

    setpoints = [0.01, None]

    # Get other inputs
    setpoint_times = square_wave_time_array(dat.SquareEntropy.square_awg)
    sp_start, sp_fin = [U.get_data_index(setpoint_times, sp) for sp in setpoints]
    logger.debug(f'Setpoint times: {setpoints}, Setpoint indexs: {sp_start, sp_fin}')

    # Run Fits
    pp = dat.SquareEntropy.get_ProcessParams(name=None,  # Load default and modify from there
                                             setpoint_start=sp_start, setpoint_fin=sp_fin,
                                             transition_fit_func=i_sense,
                                             save_name=save_name)
    out = dat.SquareEntropy.get_Outputs(name=save_name, inputs=None, process_params=pp, overwrite=overwrite)
    dat.Entropy.get_fit(which='avg', name=save_name, data=out.average_entropy_signal, x=out.x, check_exists=False,
                        overwrite=overwrite)
    [dat.Entropy.get_fit(which='row', row=i, name=save_name,
                         data=row, x=out.x, check_exists=False,
                         overwrite=overwrite) for i, row in enumerate(out.entropy_signal)]
    return out

In [6]:
def get_deltaT(dat):
    """Returns deltaT of a given dat in mV"""
    ho1 = dat.AWG.max(0)  # 'HO1/10M' gives nA * 10
    t = dat.Logs.temps.mc

    # Datnums to search through (only thing that should be changed)
    datnums = set(range(2143, 2156))
    # datnums = set()
    # for j in range(5):
    #     datnums = datnums.union(set(range(28 * j + 1312, 28 * j + 1312 + 4 * 7 + 1)) - set([28 * j + 1312 + 4 * i for i in range(8)]))
    # datnums = list(datnums)

    dats = get_dats(datnums)

    dats = [d for d in dats if np.isclose(d.Logs.temps.mc, dat.Logs.temps.mc, rtol=0.1)]  # Get all dats where MC temp is within 10%
    bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])

    indp = np.argmin(abs(bias_lookup - ho1))
    indm = np.argmin(abs(bias_lookup + ho1))
    theta_z = np.nanmean([d.Transition.avg_fit.best_values.theta for d in dats if d.Logs.fds['HO1/10M'] == 0])

    # temp_lookup = np.array([d.Logs.temps.mc for d in dats])
    # bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])
    #
    # indp = np.argmin(temp_lookup - t + bias_lookup - ho1)
    # indm = np.argmin(temp_lookup - t + bias_lookup + ho1)
    # indz = np.argmin(temp_lookup - t + bias_lookup)

    theta_p = dats[indp].Transition.avg_fit.best_values.theta
    theta_m = dats[indm].Transition.avg_fit.best_values.theta
    # theta_z = dats[indz].Transition.avg_fit.best_values.theta
    return (theta_p + theta_m) / 2 - theta_z

In [7]:
datnums = np.sort(list(set(range(7129, 7273+1)) | set(range(7304, 7321+1)) | set(range(7292, 7302+1))))

In [8]:
dats = get_dats(list(datnums), overwrite=False)

In [9]:
for i, dat in enumerate(dats):
    print(f'{i}, {dat.datnum}:{dat.Logs.temps.mc:.2f}:{dat.Logs.fds["ESC"]}')

0, 7129:0.50:-289.92
1, 7130:0.50:-284.73
2, 7131:0.50:-279.54
3, 7132:0.50:-274.35
4, 7133:0.50:-269.17
5, 7134:0.50:-263.98
6, 7135:0.50:-258.79
7, 7136:0.50:-253.6
8, 7137:0.50:-248.41
9, 7138:0.50:-243.23
10, 7139:0.49:-238.04
11, 7140:0.50:-232.85
12, 7141:0.50:-227.66
13, 7142:0.50:-222.47
14, 7143:0.50:-217.29
15, 7144:0.50:-212.1
16, 7145:0.51:-206.91
17, 7146:0.50:-201.72
18, 7147:0.50:-196.53
19, 7148:0.50:-189.82
20, 7149:0.50:-179.75
21, 7150:0.50:-169.68
22, 7151:0.50:-159.61
23, 7152:0.50:-149.54
24, 7153:0.50:-159.91
25, 7154:0.50:-149.84
26, 7155:0.50:-139.77
27, 7156:0.50:-129.7
28, 7157:0.49:-119.63
29, 7158:0.40:-289.92
30, 7159:0.40:-284.73
31, 7160:0.40:-279.54
32, 7161:0.40:-274.35
33, 7162:0.40:-269.17
34, 7163:0.40:-263.98
35, 7164:0.40:-258.79
36, 7165:0.40:-253.6
37, 7166:0.40:-248.41
38, 7167:0.40:-243.23
39, 7168:0.40:-238.04
40, 7169:0.40:-232.85
41, 7170:0.40:-227.66
42, 7171:0.40:-222.47
43, 7172:0.40:-217.29
44, 7173:0.40:-212.1
45, 7174:0.40:-206.91
46,

In [59]:
# Θ = np.average([dat.SquareEntropy.get_fit(which='avg',which_fit='transition', transition_part='cold', check_exists=False).best_values.theta for dat in progressbar(dats)])
fit = dats[0].Transition.get_fit(which='avg', check_exists=False)
Θ = 20
params = fit.params
params.add('g', value=0, vary=False, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=True)

In [83]:
fits = [narrow_fit_trans_only(
    dat,
    500,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False)
for dat in progressbar(dats[145:-1])]

100% (28 of 28) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [84]:
thetas = [fit.best_values.theta for fit in fits]
gammas = [fit.best_values.g for fit in fits]
escs = [dat.Logs.fds["ESC"] for dat in dats][145:-1]

In [85]:
temp_escs = []
temp_thetas = []
for i, esc in enumerate(escs):
    if esc < -230:
        temp_escs.append(esc)
        temp_thetas.append(thetas[i])
thetafit = np.polyfit(temp_escs, temp_thetas, 1)

In [86]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=thetas, text=datnums[145:-1], name="Data -- Thetas", marker_color="blue"))
fig.add_trace(go.Scatter(mode='markers', x=escs, y=gammas, text=datnums[145:-1], name="Data -- Gammas", marker_color="red"))
fig.add_trace(go.Scatter(mode='lines', x=temp_escs, y=np.polyval(thetafit, temp_escs), name="Fit", marker_color="grey"))

fig.update_layout(xaxis_title='ESC /mV', yaxis_title='Theta /mV',
                      title=f'Dats {dats[145].datnum} - {dats[-1].datnum}')
fig.show(renderer="chrome")

In [70]:
fig.write_html(f'Exports/Theta_dats{dats[145].datnum}_{dats[-1].datnum}.html')

In [246]:
dats_cc = []
for dat in dats:
    if dat.Logs.fds["ESC"] == -222.47:
        dats_cc.append(dat)
thetafit = np.array([ 0.09579436, 65.78364881])
dats_cc_temps = np.array([dat.Logs.temps.mc for dat in dats_cc])
dats_cc_thetas = np.polyval(thetafit, [-179.75])[0]*(dats_cc_temps/0.1)

In [247]:
for i, dat in enumerate(dats_cc):
     print(f'{i}, {dat.datnum}:{dat.Logs.temps.mc:.2f}:{dat.Logs.fds["ESC"]}')

0, 7142:0.50:-222.47
1, 7171:0.40:-222.47
2, 7200:0.30:-222.47
3, 7229:0.20:-222.47
4, 7258:0.10:-222.47
5, 7316:0.05:-222.47


In [252]:
fig = go.Figure()

gamma_params = edit_params(new_pars, param_name='g', value=0, vary=True)
xx = []
yy = []
for i, dat in progressbar(enumerate(dats_cc)):
    theta = dats_cc_thetas[i]
    theta_range = [theta + j*0.05*theta for j in range(-5, 6)]
    fit_gammas = [narrow_fit_trans_only(dat,
                                        5000,
                                        initial_params=edit_params(gamma_params, param_name='theta', value=theta, vary=False), 
                                        fit_func=i_sense_digamma, 
                                        check_exists=False) for theta in theta_range]
#     fig.add_trace(go.Scatter(mode='lines', x=theta_range/dat.Logs.temps.mc, y=fit_gammas, name=f'T = {dat.Logs.temps.mc:.2f}K'))
    
    for i in range(len(xx)):
        pass
#         xint, yint = find_intercept(theta_range/dat.Logs.temps.mc, fit_gammas, xx[i], yy[i])
#         fig.add_trace(go.Scatter(mode='markers', x=[xint], y=[yint], marker_color="grey", showlegend=False))
    
    xx.append(theta_range/dat.Logs.temps.mc)
    yy.append(fit_gammas)
#     xfit = np.linspace(-5000,5000,1001)
#     x = dat.Transition.avg_x
#     y = dat.Transition.avg_data
#     fits.append(fit_gammas[0])
#     yfit = fit_gammas[0].eval_fit(xfit)
#     fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, marker_color='grey'))
#     fig.add_trace(go.Scatter(mode='markers', x=x, y=y, name=f'T = {dats_cc_temps[i]}'))
# fig.update_layout(xaxis_title='Theta/Tmc [mV/K]', yaxis_title='Gamma /mV',
#                       title=f'ESC = {dats_cc[0].Logs.fds["ESC"]}')
# fig.show(renderer="chrome") 

| |                 #                                 | 5 Elapsed Time: 0:00:01


In [253]:
fit_gammas

[FitInfo(params=Parameters([('amp', <Parameter 'amp', value=0.31282067608330144 +/- 0.0013, bounds=[0:inf]>), ('const', <Parameter 'const', value=7.056137537849091 +/- 8.51e-05, bounds=[-inf:inf]>), ('g', <Parameter 'g', value=22.704503686014974 +/- 0.308, bounds=[-50:1000]>), ('lin', <Parameter 'lin', value=6.784436528062265e-05 +/- 2.05e-06, bounds=[0:inf]>), ('mid', <Parameter 'mid', value=0.21290763656011735 +/- 0.104, bounds=[-inf:inf]>), ('theta', <Parameter 'theta', value=18.219742886078997 (fixed), bounds=[0.01:inf]>)]), init_params=Parameters([('amp', <Parameter 'amp', value=0.3277557114199119, bounds=[0:inf]>), ('const', <Parameter 'const', value=7.256241261322933, bounds=[-inf:inf]>), ('g', <Parameter 'g', value=0, bounds=[-50:1000]>), ('lin', <Parameter 'lin', value=6.056142522536945e-05, bounds=[0:inf]>), ('mid', <Parameter 'mid', value=0.6506405224762731, bounds=[-inf:inf]>), ('theta', <Parameter 'theta', value=18.219742886078997 (fixed), bounds=[0.01:inf]>)]), func_name=

In [223]:
fig.write_html(f'Exports/Theta_vs_gamma_dats_{dats_cc[0].datnum}_{dats_cc[-1].datnum}.html')

In [233]:
def find_intercept(x1, y1, x2, y2):
    '''The idea is to get two (linear) datasets, fit them, and return their intercept'''
    x1 = np.array(x1, dtype=np.float)
    y1 = np.array(y1, dtype=np.float)
    x2 = np.array(x2, dtype=np.float)
    y2 = np.array(y2, dtype=np.float)
    fit1 = np.polyfit(x1, y1, 1)
    fit2 = np.polyfit(x2, y2, 1)
    xint = (fit2[1] - fit1[1])/(fit1[0] - fit2[0])
    yint = np.polyval(fit1, [xint])[0]
    return (xint, yint)

In [234]:
escs = [dat.Logs.fds["ESC"] for dat in dats][0:29]
xints = []
xints_std = []

for esc in progressbar(escs):
    dats_cc = []
    for dat in dats:
        if dat.Logs.fds["ESC"] == esc:
            dats_cc.append(dat)
    thetafit = np.array([ 0.09579436, 65.78364881])
    dats_cc_temps = np.array([dat.Logs.temps.mc for dat in dats_cc])
    dats_cc_thetas = np.polyval(thetafit, [esc])[0]*(dats_cc_temps/0.1)

    gamma_params = edit_params(new_pars, param_name='g', value=0, vary=True)
    xx = []
    yy = []
    xint_sf = []
    for i, dat in enumerate(dats_cc):
        theta = dats_cc_thetas[i]
        theta_range = [theta + j*0.05*theta for j in range(-5, 6)]
        fit_gammas = [narrow_fit_trans_only(dat,
                                            5000,
                                            initial_params=edit_params(gamma_params, param_name='theta', value=theta, vary=False), 
                                            fit_func=i_sense_digamma, 
                                            check_exists=False).best_values.g for theta in theta_range]
        fig.add_trace(go.Scatter(mode='lines', x=theta_range/dat.Logs.temps.mc, y=fit_gammas, name=f'T = {dat.Logs.temps.mc:.2f}K'))

        for i in range(len(xx)):
            xint, yint = find_intercept(theta_range/dat.Logs.temps.mc, fit_gammas, xx[i], yy[i])
            xint_sf.append(xint)
            

        xx.append(theta_range/dat.Logs.temps.mc)
        yy.append(fit_gammas)
     
    xints.append(np.mean(xint_sf))
    xints_std.append(np.std(xint_sf))

100% (29 of 29) |########################| Elapsed Time: 0:00:18 Time:  0:00:18


In [235]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', 
                         x=escs, 
                         y=xints, 
                         error_y=dict(type='data', # value of error bar given in data coordinates
                                      array=xints_std,
                                      visible=True)))

fig.update_layout(xaxis_title='ESC/mV', yaxis_title='Theta/Tmc [mV/mK]',
                      title=f'')
fig.show(renderer="chrome")

In [236]:
fig.write_html(f'Exports/Theta_vs_coupling_dats_{dats[0].datnum}_{dats[-1].datnum}.html')

In [28]:
def digamma_multiple(params, xxs, yys, ts):
    '''allows for fitting multiple datasets'''
    rsf = np.array([])
    for i, origx, origy, t in zip(range(len(xxs)), xxs, yys, ts):
        x = np.copy(origx)
        y = np.copy(origy)

        start_ind = np.nanargmin(np.abs(np.add(x, 3000)))
        end_ind = np.nanargmin(np.abs(np.subtract(x, 3000)))

        x[:start_ind] = [np.nan] * start_ind
        x[end_ind:] = [np.nan] * (len(x) - end_ind)

        y[:start_ind] = [np.nan] * start_ind
        y[end_ind:] = [np.nan] * (len(y) - end_ind)
        
        rsf = np.concatenate((rsf, y - i_sense_digamma_amplin(x, params[f'mid{i}'], params['g'], params['theta']*t, params[f'amp{i}'], params[f'lin{i}'], params[f'const{i}'], params[f'amplin{i}'])))  
    return rsf

In [12]:
from lmfit import minimize
from lmfit import Parameters

In [75]:
amp=0.26347
const=7.0563
g=50
lin=1.3592e-05
mid=0.11749
theta=30.366

escs = [dat.Logs.fds["ESC"] for dat in dats][0:29]
thetas = []
fits = []
for esc in progressbar(escs):
    dats_cc = []
    for dat in dats:
        if np.isclose(dat.Logs.fds["ESC"], esc, atol=0.6): #and (np.isclose(0.05, dat.Logs.temps.mc, atol=0.05) 
#                                            or np.isclose(0.1, dat.Logs.temps.mc, atol=0.05)
#                                            or np.isclose(0.2, dat.Logs.temps.mc, atol=0.05)
#                                            or np.isclose(0.3, dat.Logs.temps.mc, atol=0.05)):
            dats_cc.append(dat)

    params = Parameters()
    # add with tuples: (NAME VALUE VARY MIN  MAX  EXPR  BRUTE_STEP)
    for i in range(len(dats_cc)):
        params.add_many((f'amp{i}', amp, True, 0, None, None, None),
                        (f'const{i}', const, True, None, None, None, None),
                        (f'lin{i}', lin, True, None, None, None, None),
                        (f'mid{i}', mid, True, None, None, None, None), 
                        (f'amplin{i}', lin, True, None, None, None, None))

    params.add_many(('theta', theta, True, 0, None, None, None),
                    ('g', g, True, 0, None, None, None))

    if esc > -180:
        xxs = [dat.Transition.avg_x for dat in dats_cc]
        yys = [np.nanmean(dat.Transition.data, axis=0) for dat in dats_cc]
    else:
        xxs = [dat.Transition.avg_x for dat in dats_cc]
        yys = [dat.Transition.avg_data for dat in dats_cc]
    
    ts = [dat.Logs.temps.mc for dat in dats_cc]

    args = (xxs, yys, ts)

    res = minimize(digamma_multiple, params, 
             args = args,
             nan_policy='omit')
    thetas.append(res.params['theta'].value*0.1)
    fits.append(res.params)
    
    fig = go.Figure()
    for i, dat in enumerate(dats_cc):
        params = res.params
        if esc > -180:
            xx = dat.Transition.avg_x
            yy = np.nanmean(dat.Transition.data, axis=0)
        else:
            xx = dat.Transition.avg_x
            yy = dat.Transition.avg_data
        t = dat.Logs.temps.mc
        xfit = np.linspace(-3000,3000,1001)
        yfit = i_sense_digamma_amplin(xfit, params[f'mid{i}'], params['g'], params['theta']*t, params[f'amp{i}'], params[f'lin{i}'], params[f'const{i}'], params[f'amplin{i}'])
        fig.add_trace(go.Scatter(mode='markers', x=xx, y=yy, name=f'T = {dat.Logs.temps.mc:.2f}, {dat.datnum}d'))
        fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{dat.datnum}f', marker_color='grey'))
    
    fig.update_layout(xaxis_title='ACC*1000 /mV', yaxis_title='Isense /nA',
                      title=f'ESC = {esc}')

    if esc > -180:
        pass
#     fig.show(renderer="chrome")
#     fig.write_html(f'Exports/All_transitions_dats_{dats_cc[0].datnum}_{dats_cc[-1].datnum}_esc_{esc}.html')

100% (29 of 29) |########################| Elapsed Time: 0:01:21 Time:  0:01:21


In [45]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs, 
                         y=thetas,
                         error_y=dict(type='data', # value of error bar given in data coordinates
                                      array=np.array([f['theta'].stderr for f in fits], dtype=np.float),
                                      visible=True)))

fig.update_layout(xaxis_title='ESC/mV', yaxis_title='Theta /mV',
                      title=f'Lever Arm vs Coupling gate')

fig.show(renderer="chrome")

In [31]:
fig.write_html(f'Exports/Theta_vs_coupling_dats_{dats[0].datnum}_{dats[-1].datnum}_simul_fitting.html')

In [64]:
gammas = [f['g'].value for f in fits]

fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs, 
                         y=gammas,
                         error_y=dict(type='data', # value of error bar given in data coordinates
                                      array=np.array([f['g'].stderr for f in fits], dtype=np.float),
                                      visible=True)))

fig.update_layout(xaxis_title='ESC/mV', yaxis_title='Gamma /mV',
                      title=f'Gamma vs Coupling gate')

fig.show(renderer="chrome")

In [33]:
fig.write_html(f'Exports/Gamma_vs_coupling_dats_{dats[0].datnum}_{dats[-1].datnum}_simul_fitting.html')

In [70]:
fig = go.Figure()

data = []
escs_more = []
err = []
p = 'amplin'
for fit, esc in zip(fits, escs):
    dats_cc = []
    for dat in dats:
        if np.isclose(dat.Logs.fds["ESC"], esc, atol=0.6):
            dats_cc.append(dat)
            
    for i in range(len(dats_cc)):
        data.append(fit[p + f'{i}'].value)
        escs_more.append(esc)
        err.append(fit[p + f'{i}'].stderr)

fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs_more, 
                         y=data,
                         error_y=dict(type='data', # value of error bar given in data coordinates
                                      array=np.array(err, dtype=np.float),
                                      visible=True)))


fig.update_layout(xaxis_title='ESC/mV', yaxis_title=p,
                      title= p + f' vs Coupling gate')

fig.show(renderer="chrome")

In [69]:
fits[0].keys()

odict_keys(['amp0', 'const0', 'lin0', 'mid0', 'amplin0', 'amp1', 'const1', 'lin1', 'mid1', 'amplin1', 'amp2', 'const2', 'lin2', 'mid2', 'amplin2', 'amp3', 'const3', 'lin3', 'mid3', 'amplin3', 'amp4', 'const4', 'lin4', 'mid4', 'amplin4', 'theta', 'g'])

In [892]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=ips, text=datnums, y=deltaT, name="Entropy fits"))

fig.update_layout(xaxis_title='IP1*200 /mV', yaxis_title='deltaT /mV',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [186]:
np.mean(deltaT[0:4])

0.9546627984681944

In [ ]:
fig.write_html(f'Exports/dT_03_17_dats{datnums[0]}_{datnums[-1]}.html')

In [870]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=ips, y=amp_digamma_cold, text=datnums, name="Transition fits"))

fig.update_layout(xaxis_title='IP1*200 /mV', yaxis_title='Amplitude /mV',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [122]:
np.mean(amp_digamma_cold[0:4])

0.36157357430913073

In [1153]:
fig.write_html(f'Exports/Ampl_03_17_dats{datnums[0]}_{datnums[-1]}.html')

In [871]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=ips, y=theta_digamma_, text=datnums, name="Transition fits"))

fig.update_layout(xaxis_title='IP1*200 /mV', yaxis_title='Theta /mV',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="chrome")

In [893]:
fig = go.Figure()
for i, dat in progressbar(enumerate(dats)):
    x = dat.SquareEntropy.avg_x
    y = dat.SquareEntropy.default_Output.averaged
    ycold = np.mean(y[(0, 2), :], axis=0)
    yhot = np.mean(y[(1, 3), :], axis=0)
    xfit = np.linspace(-100,100,1001)
    fit = narrow_fit(
            dat,
            400,
            initial_params=new_pars, 
            fit_func=i_sense_digamma, 
            check_exists=False)
    yfit = fit.eval_fit(xfit) - fit.best_values.lin*xfit
#     fig.add_trace(go.Scatter(mode='markers', x=x, y=ycold - fit.best_values.lin*x, name=f'{datnums[i]}d_cold'))
#     fig.add_trace(go.Scatter(mode='markers', x=x, y=yhot - fit.best_values.lin*x, name=f'{datnums[i]}d_hot'))
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y[0, :] - fit.best_values.lin*x, name=f'{datnums[i]}d_v0_0'))
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y[1, :] - fit.best_values.lin*x, name=f'{datnums[i]}d_vp'))
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y[2, :] - fit.best_values.lin*x, name=f'{datnums[i]}d_v_0_1'))
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y[3, :] - fit.best_values.lin*x, name=f'{datnums[i]}d_vm'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{datnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title=dats[0].Logs.xlabel, yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |    #                                             | 10 Elapsed Time: 0:00:00


In [882]:
dats[0].SquareEntropy.square_awg.max(0)

20.0

In [85]:
fig.write_html(f'Exports/Cold_hot_transitions_plus_fit_dats{dats[0].datnum}_{dats[-1].datnum}.html')

In [872]:
ampl = [0.34 for dat in dats] #0.44
deltaT = [4.9 for dat in dats] #0.96
for i, dat in enumerate(dats):
    dat.Entropy.set_integration_info(dT=deltaT[i], amp=ampl[i], overwrite=True)

In [873]:
fig = go.Figure()
int_ents = []
int_ents_2d = []
int_ents_peaks = []
for i in range(len(dats)):
    width = dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.theta*10
#     width = g_digamma_[i]*1
    center = dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.mid
    indstart = np.argmin(np.abs(np.subtract(out[i].x, center-width)))
    indend = np.argmin(np.abs(np.subtract(out[i].x, center+width)))
    
    int_ent = integrate_entropy(out[i].average_entropy_signal, dats[i].Entropy.integration_info.sf)
    int_ents_2d.append(int_ent)
    int_ents_peaks.append(max(int_ent))
    int_ents.append(int_ent[indend] - int_ent[indstart])
    fig.add_trace(go.Scatter(mode='markers', 
                             x=np.subtract(out[i].x, mids_digamma_[i]), 
                             y=int_ent,
                             text=f'Int Ent = {int_ents[i]:.2f}',
                             name= f'dat{dats[i].datnum}, IP*200:{dats[i].Logs.fds["IP1*200"]}'))

fig.update_layout(xaxis_title=dats[0].Logs.xlabel, yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='chrome')
print(width)

56.12783727913989


In [287]:
fig.write_html(f'Exports/IntEntropy_03_22_dats{dats[0].datnum}_{dats[-1].datnum}.html')

In [874]:
ents = [dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.dS for dat in dats]
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=int_ents, 
                         text=datnums, 
                         name="Int",
                         marker=dict(color='LightSkyBlue',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=ents, 
                         text=datnums, 
                         name="Fit",
                         marker=dict(color='Brown',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=int_ents_peaks, 
                         text=datnums, 
                         name="Integrated Peak",
                         marker=dict(color='Green',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))


fig.update_layout(xaxis_title="IP1*200 /mV", yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="chrome")

In [636]:
fig.write_html(f'Exports/IntEntropyvalues_03_24_dats{datnums[0]}_{datnums[-1]}.html')

In [261]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=np.subtract(int_ents_peaks,int_ents), 
                         text=datnums, 
                         name="Peak-Int",
                         marker=dict(color='Green',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))

fig.update_layout(xaxis_title='IP1*200 /mV', yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [622]:
fig = go.Figure()
for i in range(len(dats)):
    enty = out[i].average_entropy_signal
    entx = out[i].x
    fitent = dats[i].Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).eval_fit(entx)
    ent = dats[i].Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.dS
    fig.add_trace(go.Scatter(mode='markers', 
                             x=entx,
                             y=enty,
                             name= f'dat{dats[i].datnum}, IP*200:{dats[i].Logs.fds["IP1*200"]}'))
    
    fig.add_trace(go.Scatter(mode='lines', x=entx, y=fitent, name=f'{datnums[i]}f, {ent}', marker_color='grey'))

fig.update_layout(xaxis_title=dats[0].Logs.xlabel, yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='chrome')

In [417]:
fig.write_html(f'Exports/FitEntropy_03_23_dats{datnums[0]}_{datnums[-1]}.html')

In [801]:
entz_2d = [out[i].average_entropy_signal for i in range(len(dats))]

In [839]:
fig = go.Figure()
fig.add_trace(go.Heatmap(
                    z=int_ents_2d, x=out[0].x, y=[ips[j] for j in range(len(dats))],
                    colorscale='RdBu', zmid=0, zmax=2, zmin=-2)
             )
fig.update_layout(xaxis_title='ACC*100, IP1*200 /mV', yaxis_title='IP1*200',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}, IS = {dats[0].Logs.bds["IS"]}')
fig.show(renderer='chrome')

In [639]:
fig.write_html(f'Exports/dndt_2d_dats{datnums[0]}_{datnums[-1]}.html')

In [623]:
centers = [dat.SquareEntropy.get_fit(data=dat.Data.get_data('Experiment Copy/cscurrent_2d')[0],
                                     name="Centers",
                                     x=dat.Data.get_data('Experiment Copy/x_array'),
                                     which_fit='transition', 
                                     check_exists=False).best_values.mid
for dat in progressbar(dats)]

100% (17 of 17) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


In [624]:
occ_data = get_dat(5055)
data = occ_data.Data.get_data('Experiment Copy/cscurrent_2d')
x=occ_data.Data.get_data('Experiment Copy/x_array')
y=occ_data.Data.get_data('Experiment Copy/y_array')
accs = [dat.Logs.fds["ACC*100"] for dat in dats]

In [625]:
fig = go.Figure()
ips_other = np.array(ips)
fig.add_trace(go.Heatmap(
                    z=np.diff(data), x=x[:-1], y=y))
fig.add_trace(go.Scatter(mode='markers', 
                         x=centers, y=ips_other, 
                         text=[dat.datnum for dat in dats],
                         marker_color="White"))
fig.update_layout(xaxis_title='ACC*100 /mV', yaxis_title='IP1*200',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}, dat{occ_data.datnum}')
fig.show(renderer='chrome')

In [415]:
fig.write_html(f'Exports/Along_occ_03_21_{dats[0].datnum}_{dats[-1].datnum}.html')

In [353]:
ips

[-700.0, -600.0, -500.0, -400.0, -300.0, -200.0, -100.0, 0.0, 100.0]